# 正則化 (Regularization)


In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.2の大きさのノイズが乗ったデータを考えます。データ数は30にします。
早期終了の例と同様に、scikit learningの関数を使ってデータをトレーニング用と検証用に分割します。

In [ ]:
np.random.seed(0)

nSample = 30

x = np.linspace(-np.pi, np.pi, nSample)
t0 = np.sin(x)
t = t0 + 0.2 * np.random.randn(nSample)

from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.2, random_state=42)

# For plot
x_grid = np.linspace(-np.pi, np.pi, 100)

# numpy -> torch.tensor
from torch import from_numpy
x_train_tensor = from_numpy(x_train).float().unsqueeze(-1)
t_train_tensor = from_numpy(t_train).float().unsqueeze(-1)
x_valid_tensor = from_numpy(x_valid).float().unsqueeze(-1)
t_valid_tensor = from_numpy(t_valid).float().unsqueeze(-1)
x_grid_tensor = from_numpy(x_grid).float().unsqueeze(-1)

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、隠れ層が5層、ノード数が128の隠れ層を4層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.optim import Adam

# モデルの定義
model = Sequential(
    Linear(in_features=1, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = MSELoss()
optimizer = Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in range(num_epochs):
    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
plt.plot(x_grid, model(x_grid_tensor).detach().numpy(), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train_history, label='loss (train)')
plt.plot(loss_valid_history, label='loss (valid)')
plt.legend()
plt.show()


## L2 正則化
過学習の問題は、トレーニングサンプルの数が少なすぎる時、もしくはモデルの表現能力が高すぎる、パラメータの自由度が大きすぎると起こります。
校舎を抑制するため、パラメータの自由度を適度に制約してやることで過学習を抑えることができます。

正則化(Regularization)は、パラメータに制限をかけることで、過学習を抑制します。

代表的なものはL2 正則、もしくは *荷重減衰(weight decay)* と呼ばれる手法で、重みパラメータの2乗和
$$
L_{\text L2} = \sum_i w_i^2
$$
の大きさでペナルティーをかけることで、重みパラメータが過度に大きくならないようにします。

(実はこの制約のかけ方は、$w_i$の事前分布としてガウス分布を仮定すると、導出されます。興味のあるかたは調べてみてください。)



PyTorchでは、以下のようにしてL2ロスを実装できます。

```python
l2_lambda = 0.01
l2 = torch.tensor(0., requires_grad=True)
for weights in model.parameters():
    l2 = l2 + weights.square().sum()
loss_reg = loss + l2_lambda * l2
```

`l2_labmda`の大きさを変更することで、ペナルティの大きさを調整することもできます。 

例えば、`l2_labmda = 0.01`のようにした場合、
$$
L_{\text total} = L + 0.01 \cdot \sum_i w_i^2
$$
のようなロス関数を最小化することになります。

実際に学習をさせてみましょう。

In [ ]:

from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.optim import Adam
from torch import tensor

# モデルの定義
model = Sequential(
    Linear(in_features=1, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = MSELoss()
optimizer = Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
l2_lambda = 0.01
for i_epoch in range(num_epochs):
    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # L2正則化項の追加
    l2 = tensor(0., requires_grad=True)
    for weights in model.parameters():
        l2 = l2 + weights.square().sum()
    loss_reg = loss + l2_lambda * l2

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss_reg.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    # L2を除いたロス関数の推移を取得します
    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
plt.plot(x_grid, model(x_grid_tensor).detach().numpy(), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train_history, label='loss (train)')
plt.plot(loss_valid_history, label='loss (valid)')
plt.legend()
plt.show()


正則化によって過学習が抑制されてます。また検証用データに誤差関数の値が十分に小さく抑えられているはずです。

PyTorchではoptimizerの引数に値を指定することでもL2正則化を入れることができます。L2正則化を荷重減衰(weight decay)と呼ぶこともありますが、optimizerの引数の`weight decay`を使ってL2正則化を適用できます。

```python
optimizer = Adam(model.parameters(), weight_decay=0.01)
```

In [ ]:

from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.optim import Adam
from torch import tensor

# モデルの定義
model = Sequential(
    Linear(in_features=1, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), weight_decay=0.01)

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in range(num_epochs):
    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    # L2を除いたロス関数の推移を取得します
    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
plt.plot(x_grid, model(x_grid_tensor).detach().numpy(), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train_history, label='loss (train)')
plt.plot(loss_valid_history, label='loss (valid)')
plt.legend()
plt.show()


## L1正則化
もう一つよく使われる正則化手法として、L1 正則化があります。
こちらは、Lasso とも呼ばれることがあります。

L1 正則化では、
$$
L_{\text L1} = \sum_i |w_i|
$$
の形で誤差関数にペナルティーを与えます。
L1正則化は、L2正則化と比較して、$w_i$が0になりやすいという特徴があります。
このことから、モデルパラメータを疎にする目的でこの手法が使われることもあります。

PyTorchでもL2正則化と同様にして実装できます。

```python
l1_lambda = 0.001
l1 = tensor(0., requires_grad=True)
for weights in model.parameters():
    l1 = l1 + weights.abs().sum()
loss_reg = loss + l1_lambda * l1
```


In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.optim import Adam
from torch import tensor

# モデルの定義
model = Sequential(
    Linear(in_features=1, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = MSELoss()
optimizer = Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
l1_lambda = 0.001
for i_epoch in range(num_epochs):
    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # L1正則化項の追加
    l1 = tensor(0., requires_grad=True)
    for weights in model.parameters():
        l1 = l1 + weights.abs().sum()
    loss_reg = loss + l1_lambda * l1

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss_reg.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    # L2を除いたロス関数の推移を取得します
    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
plt.plot(x_grid, model(x_grid_tensor).detach().numpy(), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train_history, label='loss (train)')
plt.plot(loss_valid_history, label='loss (valid)')
plt.legend()
plt.show()


また、L1正則化とL2正則化の両方をかけることもできます。
$$
L_{\text total} = L + \lambda_{L1} \cdot \sum_i |w_i| + \lambda_{L2} \cdot \sum_i w_i^2
$$

```python
l1_labmda = 0.001
l2_labmda = 0.01
l1 = tensor(0., requires_grad=True)
l2 = tensor(0., requires_grad=True)
for weights in model.parameters():
    l1 = l1 + weights.abs().sum()
    l2 = l2 + weights.square().sum()
loss_reg = loss + l1_lambda * l1 + l2_lambda * l2
```

L1, L2正則化の大きさを変化させて、モデルの予測がどのように変化するかを調べてみましょう。